In [3]:
import warnings 
warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
import numpy as np
import pandas as pd
from neuromaps.images import load_data, load_gifti, annot_to_gifti, relabel_gifti, construct_shape_gii
from neuromaps.resampling import resample_images
from neuromaps.nulls import alexander_bloch, burt2020
from neuromaps.parcellate import Parcellater
from scipy.stats import pearsonr
from neuromaps import transforms 
from neuromaps.stats import compare_images
from neuromaps.nulls import hungarian
from statsmodels.stats.multitest import multipletests
import pickle 


In [4]:
# define path
base_path = os.path.dirname(os.getcwd())

In [5]:
# load turku 
turku_parc = np.load(os.path.join(base_path, 'data' 'turku_parc.npy'))

# load enigma
enigma_file='ENIGMA_S32_partial_correlation_between_cortical_thickness_and_chlorpromazine_equivalents.csv'
enigmamap = pd.read_csv(os.path.join(base_path, 'data',enigma_file))
enigmamap.drop([68, 69], inplace=True)  # Remove the last two rows
enigma_parc = enigmamap['partial_r'].to_numpy()

# load neurosynth
neurosynth = pd.read_csv(os.path.join(base_path, 'data', 'atl-desikankilliany_res-83_neurosynth.csv') )
rois = pd.read_csv(os.path.join(base_path, 'parcellations', 'atlas-desikankilliany.csv'))
neurosynth['structure'] = rois['structure']
neurosynth = neurosynth[neurosynth['structure'].str.match('cortex')]
neurosynth.drop(['Unnamed: 0', 'structure'], axis=1,  inplace=True)

# term that does not match
categ = pd.read_csv(os.path.join(base_path, 'data', 'neurosynth_categories.csv'), header=None)
categ = categ.iloc[:,0].to_list()
idx = np.array([n in categ for n in neurosynth.columns], dtype=bool)
neurosynth = neurosynth.iloc[:,idx]

# load rois 
rois = pd.read_csv(os.path.join(base_path, 'parcellations' ,'atlas-desikankilliany.csv'))


# get spins
spins = pd.read_csv(os.path.join(base_path, 'parcellations','spins_hungarian_aparc+aseg_ctx.csv'), header=None)
nspins = spins.values.shape[1]

# get parcellation
def load_atlas(base_path):
    """
    Load different parcellation files.
    """    
    atlas= {
        'dk_fsaverage_10k': (
            os.path.join(base_path, 'parcellations', 'atlas-desikankilliany_space-fsaverage_den-10k_hemi-L.label.gii.gz'),
            os.path.join(base_path, 'parcellations', 'atlas-desikankilliany_space-fsaverage_den-10k_hemi-R.label.gii.gz')
    ),
    'dk_fsaverage_164k': (
            os.path.join(base_path, 'parcellations', 'atlas-desikankilliany_space-fsaverage_den-164k_hemi-L.aparc-1.annot'),
            os.path.join(base_path, 'parcellations', 'atlas-desikankilliany_space-fsaverage_den-164k_hemi-R.aparc-1.annot')
    ),
    'dk_mni': os.path.join(base_path, 'parcellations', 'atlas-desikankilliany_space-MNI_res-1mm.nii.gz')
    }
    return (atlas)

atlas = load_atlas(base_path)

In [6]:
def calc_corrs_neurosynth(parc):
    """
    Calculate correlations & nulls between neurosynth &
    parcellated (parc) antipsychotic effects on cortical thickness
    """

   # initialize
    nulls = dict([])
    corrs = dict([])
    
    for column in neurosynth:

        # empirical correlation between annotations and parc
        rho = pearsonr(neurosynth[column].to_numpy(), parc)[0]

        # get 10k rotations
        rotated = hungarian(data=neurosynth[column], n_perm=10000, spins=spins, parcellation=atlas['dk_fsaverage_164k'])

        # get null
        n = np.zeros((nspins, ))
        for i in range(nspins):
            n[i] = pearsonr(parc, rotated[:,i])[0]    

        # get p-value
        pspin = (1 + sum(abs(n) > abs(rho ))) / (nspins + 1)

        # store, multiply by -1 to make more intuitive, because smaller p-value/rho means bigger effect  
        corrs[column] = ( (-1 * rho, pspin ) )
        nulls[column] = n

    return(nulls, corrs)

nulls_turku, corrs_turku = calc_corrs_neurosynth(turku_parc)
nulls_enigma, corrs_enigma = calc_corrs_neurosynth(enigma_parc)

In [59]:
# categorize, calculate means, permutate
# code for this bit modified from here https://github.com/netneurolab/bazinet_assortativity/blob/master/main_script.py

# Load information about categories
def get_neurosynth_results(corrs): 
    
    # initialize
    neurosynth_results = {} 
    
    neurosynth_results['categories_labels'] = [ 'action','learn./memory', 'other', 'emotion', 'attention',
                                               'reason/deci. making', 'exec./cog. control', 
                                               'social function', 'perception', 'motivation', 'language' ]                                          
    
    # get categories for each term
    neurosyn_categories = pd.read_csv(path + "data/neurosynth_categories.csv", header=None).values[:, 1].astype(int)
    neurosynth_results['categories'] = neurosyn_categories
    neurosynth_results['rho'] = np.array([value[0] for value in corrs.values()])
    neurosynth_results['pspin'] = np.array([value[1] for value in corrs.values()])
    
    # Compute mean of each category
    neurosynth_results['r_categories'] =  np.array([neurosynth_results['rho'][neurosyn_categories == 1+i].mean() 
                                                   for i in range(11)])
    
    # Compute mean after permuting categories
    neurosynth_results['r_categories_perm'] = np.zeros((11, 10000))
    for i in range(11):
        for j in range(10000):
            perm = np.random.permutation(neurosyn_categories)
            r_perm_mean = neurosynth_results['rho'][perm == i+1].mean()
            neurosynth_results['r_categories_perm'][i, j] = r_perm_mean

    # Compute p-values by comparing means and permuted means 
    neurosynth_results['r_categories_p'] = np.zeros((11))
    for i in range(11):
        neurosynth_results['r_categories_p'][i] = (1 + sum( abs(neurosynth_results['r_categories_perm'][i, :]) > abs(neurosynth_results['r_categories'][i])))/ 10001
    
    # multiple comparisons 
    _, neurosynth_results['r_categories_p_fdr'], _, _ = multipletests(
    neurosynth_results['r_categories_p'], method='fdr_by')
    
    return(neurosynth_results)

neurosynth_results_turku = get_neurosynth_results(corrs_turku)
neurosynth_results_enigma = get_neurosynth_results(corrs_enigma)

In [60]:
# save
with open(path + 'data/neurosynth_results_turku.pkl', 'wb') as f:
    # dump the dictionary into the file
    pickle.dump(neurosynth_results_turku, f)
    
with open(path + 'data/neurosynth_results_enigma.pkl', 'wb') as f:
    # dump the dictionary into the file
    pickle.dump(neurosynth_results_enigma, f)

In [61]:
# get the top 10% of posivitive and top 10 % negative correlations
def topbottom(corrs, nulls):
    df1 = pd.DataFrame.from_dict(corrs, orient="index", columns=["rho", "pspin"])
    df1.sort_values(by="rho", inplace=True)
    df1 = pd.concat([df1.iloc[:12], df1.iloc[-12:]])
    index_list = list(df1.index.values)
    new_corrs = df1.T.to_dict(orient='list')
    
    df2 = pd.DataFrame.from_dict(nulls)
    df2 = df2[index_list]
    new_nulls = df2.to_dict(orient='list')
    return(new_corrs, new_nulls)

corrs_turku_tb, nulls_turku_tb = topbottom(corrs_turku, nulls_turku)
corrs_enigma_tb, nulls_enigma_tb = topbottom(corrs_enigma, nulls_enigma)

np.savez(path + 'data/corrs_turku_neurosynth_tb.npz', **corrs_turku_tb)
np.savez(path + 'data/nulls_turku_neurosynth_tb.npz', **nulls_turku_tb)
np.savez(path + 'data/nulls_enigma_neurosynth_tb.npz', **nulls_enigma_tb)
np.savez(path + 'data/corrs_enigma_neurosynth_tb.npz', **corrs_enigma_tb)
